In [7]:
import argparse
import os
import numpy as np
import time

from PIL import Image
import pandas as pd

In [169]:
#read 2d traits
exp_path = "/media/lietang/easystore1/RoAD/exp16"
date_file = os.path.join(exp_path, "experiment_date.txt")
dates = open(date_file).read().splitlines()

label_file = os.path.join(exp_path, "experiment_label.txt")
labels = open(label_file).read().splitlines()

traits_2d_csv_path = os.path.join(exp_path, '2d_image_deeplab.csv')
traits_3d_csv_path = os.path.join(exp_path, '3d_images', 'traits_holistic_3d.csv')
traits_leaf_csv_path = os.path.join(exp_path, 'leaf_seg_eigen_dis2', 'individual_leaf_traits.csv')

In [170]:
traits_2d = pd.read_csv(traits_2d_csv_path, sep='\t')
print(traits_2d) 

                  label       date    area_2d  convex_area_2d  bbox_area_2d  \
0     N113-1_W_135.92_1   2020-7-7   0.316305        0.762921      1.115959   
1     N113-1_W_135.92_1   2020-7-8   0.379775        0.928920      1.245224   
2     N113-1_W_135.92_1   2020-7-9   0.461961        1.064113      1.457721   
3     N113-1_W_135.92_1  2020-7-10   0.603780        1.289979      1.886203   
4     N113-1_W_135.92_1  2020-7-11   0.709680        1.426916      2.350954   
...                 ...        ...        ...             ...           ...   
7853     WT-3_W_65.54_4   2020-8-4  11.044506       17.422328     25.916985   
7854     WT-3_W_65.54_4   2020-8-5  11.748374       17.901377     26.423119   
7855     WT-3_W_65.54_4   2020-8-6  12.239048       18.550958     26.883103   
7856     WT-3_W_65.54_4   2020-8-7  13.040213       19.279237     27.619985   
7857     WT-3_W_65.54_4   2020-8-8  14.690437       22.059254     29.939552   

      perimeter_2d  major_axis_length_2d  minor_axi

In [171]:
traits_3d = pd.read_csv(traits_3d_csv_path, sep=',', index_col=False)
print(traits_3d) 

                 pot_id sample_date  num_size_3d  plant_height_3d  \
0     N113-1_W_135.92_1    2020-7-7          237         0.377778   
1     N113-1_W_135.92_1    2020-7-8          327         0.406507   
2     N113-1_W_135.92_1    2020-7-9          405         0.444382   
3     N113-1_W_135.92_1   2020-7-10          412         0.482981   
4     N113-1_W_135.92_1   2020-7-11          521         0.509181   
...                 ...         ...          ...              ...   
7853     WT-3_W_65.54_4    2020-8-4        19102         1.089870   
7854     WT-3_W_65.54_4    2020-8-5        19784         1.241880   
7855     WT-3_W_65.54_4    2020-8-6        20693         0.988859   
7856     WT-3_W_65.54_4    2020-8-7        22076         0.883738   
7857     WT-3_W_65.54_4    2020-8-8        24590         1.636950   

      holistic_aspect_ratio_3d  length_of_boundingbox_3d  \
0                      5.64612                   1.00587   
1                      3.82908                   1.

In [172]:
new_df = pd.merge(traits_3d, traits_2d,  how='left', left_on=['pot_id','sample_date'], right_on = ['label','date'])
del new_df['label']
del new_df['date']

print(new_df)
# output_path = os.path.join(exp_path, "traits_all0.csv")
# new_df.to_csv(output_path, index = False)



                 pot_id sample_date  num_size_3d  plant_height_3d  \
0     N113-1_W_135.92_1    2020-7-7          237         0.377778   
1     N113-1_W_135.92_1    2020-7-8          327         0.406507   
2     N113-1_W_135.92_1    2020-7-9          405         0.444382   
3     N113-1_W_135.92_1   2020-7-10          412         0.482981   
4     N113-1_W_135.92_1   2020-7-11          521         0.509181   
...                 ...         ...          ...              ...   
7853     WT-3_W_65.54_4    2020-8-4        19102         1.089870   
7854     WT-3_W_65.54_4    2020-8-5        19784         1.241880   
7855     WT-3_W_65.54_4    2020-8-6        20693         0.988859   
7856     WT-3_W_65.54_4    2020-8-7        22076         0.883738   
7857     WT-3_W_65.54_4    2020-8-8        24590         1.636950   

      holistic_aspect_ratio_3d  length_of_boundingbox_3d  \
0                      5.64612                   1.00587   
1                      3.82908                   1.

In [173]:
traits_list_prop = ['label','date','weight_before_watering','weight_after_watering','water_added','DAS']

traits_props = pd.DataFrame( columns = traits_list_prop) 

for label in labels:
    print(label)
#     if label != 'S312-3_W_66.85_2':
#         continue
    for date in dates:
#         if date != '2021-2-22':
#             continue
        print(date)

        #add weight properties
        weight_path = os.path.join(exp_path, date, label, label + 'parameters.txt')
        if not os.path.exists(weight_path):
            continue
        with open(weight_path) as f:
            content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.strip() for x in content] 
#         print(content)

        wbw = float(content[3])
        waw = float(content[5])
        
#         df.loc[len(df)]=['8/19/2014','Jun','Fly','98765'] 
        DAS = content[6].split(":")[-1]
        if int(DAS)>100:
            DAS = 0
        if (wbw < 1000) & (waw < 1000):
            traits_props.loc[len(traits_props)]= [label, date, wbw, waw, waw-wbw, DAS]
        else:
            traits_props.loc[len(traits_props)]= [label, date, 0, 0, 0, DAS]
            
        count = count + 1
#         if count>10:
#             break
#     break
    
print(traits_props)


N113-1_W_135.92_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
N113-1_W_135.92_2
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
N113-1_W_135.92_3
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
202

2020-8-9
2020-8-10
2020-8-11
N35-1_W_135.92_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
N35-1_W_135.92_2
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
N35-1_W_135.92_3
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-

2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
N35-3_W_65.54_4
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S116-1_W_135.92_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S116-1_W_135.92_2
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2

2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S116-3_W_65.54_3
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S116-3_W_65.54_4
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S19-1_W_135.92_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-

2020-8-10
2020-8-11
S19-3_W_65.54_2
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S19-3_W_65.54_3
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S19-3_W_65.54_4
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30

...

2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S34-3_W_135.92_4
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S34-3_W_65.54_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S34-3_W_65.54_2
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2

2020-8-8
2020-8-9
2020-8-10
2020-8-11
S40-3_W_135.92_3
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S40-3_W_135.92_4
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S40-3_W_65.54_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28

2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S93-3_W_135.92_3
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S93-3_W_135.92_4
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S93-3_W_65.54_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7

2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S96-3_W_135.92_2
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
S96-3_W_135.92_3
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10

2020-8-11
WT-2_W_65.54_4
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
WT-3_W_135.92_1
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31
2020-8-1
2020-8-2
2020-8-3
2020-8-4
2020-8-5
2020-8-6
2020-8-7
2020-8-8
2020-8-9
2020-8-10
2020-8-11
WT-3_W_135.92_2
2020-7-7
2020-7-8
2020-7-9
2020-7-10
2020-7-11
2020-7-12
2020-7-13
2020-7-14
2020-7-15
2020-7-16
2020-7-17
2020-7-18
2020-7-19
2020-7-20
2020-7-21
2020-7-22
2020-7-23
2020-7-24
2020-7-25
2020-7-26
2020-7-27
2020-7-28
2020-7-29
2020-7-30
2020-7-31


In [174]:
traits_all = pd.merge(new_df, traits_props,  how='left', left_on=['pot_id','sample_date'], right_on = ['label','date'])
del traits_all['label']
del traits_all['date']

print(traits_all)
output_path = os.path.join(exp_path, "traits_all.csv")
traits_all.to_csv(output_path, index = False)

                 pot_id sample_date  num_size_3d  plant_height_3d  \
0     N113-1_W_135.92_1    2020-7-7          237         0.377778   
1     N113-1_W_135.92_1    2020-7-8          327         0.406507   
2     N113-1_W_135.92_1    2020-7-9          405         0.444382   
3     N113-1_W_135.92_1   2020-7-10          412         0.482981   
4     N113-1_W_135.92_1   2020-7-11          521         0.509181   
...                 ...         ...          ...              ...   
7853     WT-3_W_65.54_4    2020-8-4        19102         1.089870   
7854     WT-3_W_65.54_4    2020-8-5        19784         1.241880   
7855     WT-3_W_65.54_4    2020-8-6        20693         0.988859   
7856     WT-3_W_65.54_4    2020-8-7        22076         0.883738   
7857     WT-3_W_65.54_4    2020-8-8        24590         1.636950   

      holistic_aspect_ratio_3d  length_of_boundingbox_3d  \
0                      5.64612                   1.00587   
1                      3.82908                   1.

In [ ]:
delete below

In [115]:


output_path = os.path.join(exp_path, "traits_all.csv")
count = 0

traits_list_3d = ['pot_id','sample_date','num_size_3d','plant_height_3d','holistic_aspect_ratio_3d',
                  'length_of_boundingbox_3d','width_of_boundingbox_3d','height_of_boundingbox_3d',
                  'area_3d','convex_area_3d','holistic_area_convexity_3d',
                  'convex_hull_volume_3d','boundingbox_volume_3d','volume_3d','solidity_3d',
                  'holistic_curvature_3d'
                 ]
traits_list_2d = ['area_2d','convex_area_2d','bbox_area_2d','perimeter_2d', 
                  'major_axis_length_2d','minor_axis_length_2d','solidity_2d','eccentricity_2d','abs_area_2d',
                  'R_healthy', 'G_healthy', 'B_healthy', 'H_healthy', 'S_healthy', 'V_healthy', 'area_healthy_2d',
                  'R_wilted', 'G_wilted', 'B_wilted', 'H_wilted', 'S_wilted', 'V_wilted', 'area_wilted_2d',
                  'R_dry', 'G_dry', 'B_dry', 'H_dry', 'S_dry', 'V_dry', 'area_dry_2d',
                  'abs_day'
          ]

traits_list_prop = ['weight_before_watering','weight_after_watering','water_added','DAS']

traits = np.append(traits_list_3d, traits_list_2d)
traits = np.append(traits, traits_list_prop)

df = pd.DataFrame( columns = traits)

for label in labels:
#     print(label)
    if label != 'S312-3_W_66.85_2':
        continue
    for date in dates:
        if date != '2021-2-22':
            continue
#         print(date)
        traits_2d_today = traits_2d.loc[(traits_2d['label'] == label) & (traits_2d['date'] == date)]
        traits_3d_today = traits_3d.loc[(traits_3d['pot_id'] == label) & (traits_3d['sample_date'] == date)]
        traits_today = pd.concat([traits_3d_today, traits_2d_today.iloc[:, 2:]], axis=1)
               
        print(traits_2d_today.shape)
        print(traits_3d_today.shape)
        print(traits_today.shape)
        print(traits_2d_today.iloc[:, 2:])
        
        if (traits_2d_today.shape[0]<1 or traits_3d_today.shape[0]<1):
            continue
        
        #add weight properties
        weight_path = os.path.join(exp_path, date, label, label + 'parameters.txt')
        if not os.path.exists(weight_path):
            continue
        with open(weight_path) as f:
            content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.strip() for x in content] 
#         print(content)

        wbw = float(content[3])
        waw = float(content[5])
        if (wbw < 1000) & (waw < 1000):
            traits_today['weight_before_watering'] = wbw
            traits_today['weight_after_watering'] = waw
            traits_today['water_added'] = waw - wbw
        else:
            traits_today['weight_before_watering'] = 0
            traits_today['weight_after_watering'] = 0
            traits_today['water_added'] = 0
            
        traits_today['DAS'] = content[6].split(":")[-1] #MAOHAO HOUMIAN DE
        
        df = df.append(traits_today)
        
#         print(traits_2d_today)
#         print(traits_3d_today)

#         count = count + 1
#         if count>300:
#             break
#     break
    
print(df)
df.to_csv(output_path, sep='\t', index = False)

(1, 32)
(1, 16)
(2, 46)
       area_2d  convex_area_2d  bbox_area_2d  perimeter_2d  \
6233  0.260042        0.569372      0.694684      4.692243   

      major_axis_length_2d  minor_axis_length_2d  solidity_2d  \
6233              1.029139              0.856364     0.456717   

      eccentricity_2d  abs_area_2d  R_healthy  ...  S_wilted  V_wilted  \
6233           0.5546          0.0  64.556549  ...       0.0       0.0   

      area_wilted_2d  R_dry  G_dry  B_dry  H_dry  S_dry  V_dry  area_dry_2d  
6233             0.0    0.0    0.0    0.0    0.0    0.0    0.0          0.0  

[1 rows x 30 columns]
                pot_id sample_date  num_size_3d  plant_height_3d  \
6182  S312-3_W_66.85_2   2021-2-22        242.0         0.401669   
6233               NaN         NaN          NaN              NaN   

      holistic_aspect_ratio_3d  length_of_boundingbox_3d  \
6182                   3.12863                   1.14966   
6233                       NaN                       NaN   

      

In [24]:
traits_3d.loc[(traits_3d['pot_id'] == label) & (traits_3d['sample_date'] == date)]


,pot_id,sample_date,num_size_3d,plant_height_3d,holistic_aspect_ratio_3d,length_of_boundingbox_3d,width_of_boundingbox_3d,height_of_boundingbox_3d,area_3d,convex_area_3d,holistic_area_convexity_3d,convex_hull_volume_3d,boundingbox_volume_3d,volume_3d,solidity_3d,holistic_curvature_3d
0,M470-1_W_138.92_1,2021-2-22,256,0.551006,2.15059,1.01537,0.940632,0.472133,0.216719,1.96483,0.110299,0.1569,0.450927,0.03714,0.236715,0.112848
